In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import keras.backend.tensorflow_backend as K
from keras.models import load_model
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras.models import Model
from keras import losses

Using TensorFlow backend.


In [2]:
#dir = './sequence/*'
dir = './latest_sequence/bfs/*'

In [3]:
# file read
all_data = []
sequence_length = []
name = []
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [4]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [5]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [25]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [33]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    #loss2 = losses.categorical_crossentropy(y_true, y_pred)
    return loss1

In [47]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(256, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(128, return_sequences=True)(encoded)
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = LSTM(256, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder_256 = Model(inputs, decoded)
lstm_autoencoder_256.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

In [ ]:
lstm_autoencoder_256.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)

Epoch 1/500
2352/2352 [==============================] - 51s 22ms/step - loss: 35.9389
Epoch 2/500
2352/2352 [==============================] - 49s 21ms/step - loss: 35.1550
Epoch 3/500
2352/2352 [==============================] - 53s 22ms/step - loss: 35.1172
Epoch 4/500
2352/2352 [==============================] - 48s 20ms/step - loss: 36.5811
Epoch 5/500
2352/2352 [==============================] - 48s 21ms/step - loss: 35.3132
Epoch 6/500
2352/2352 [==============================] - 47s 20ms/step - loss: 35.2881
Epoch 7/500
2352/2352 [==============================] - 47s 20ms/step - loss: 35.2688
Epoch 8/500
2352/2352 [==============================] - 48s 20ms/step - loss: 35.2530
Epoch 9/500
2352/2352 [==============================] - 46s 20ms/step - loss: 35.2400
Epoch 10/500
2352/2352 [==============================] - 46s 19ms/step - loss: 35.2291
Epoch 11/500
2352/2352 [==============================] - 46s 19ms/step - loss: 35.2198
Epoch 12/500
2352/2352 [=================

2352/2352 [==============================] - 66s 28ms/step - loss: 35.0667
Epoch 95/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0661
Epoch 96/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0656
Epoch 97/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0650
Epoch 98/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0645
Epoch 99/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0639
Epoch 100/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0634
Epoch 101/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0629
Epoch 102/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0624
Epoch 103/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0619
Epoch 104/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0614
Epoch 105/500
2352/2352 [===============

2352/2352 [==============================] - 65s 28ms/step - loss: 35.0430
Epoch 187/500
2352/2352 [==============================] - 65s 27ms/step - loss: 35.0429
Epoch 188/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0428
Epoch 189/500
2352/2352 [==============================] - 65s 28ms/step - loss: 35.0427
Epoch 190/500
2352/2352 [==============================] - 65s 27ms/step - loss: 35.0426
Epoch 191/500
2352/2352 [==============================] - 65s 27ms/step - loss: 35.0426
Epoch 192/500
2352/2352 [==============================] - 65s 27ms/step - loss: 35.0425
Epoch 193/500
2352/2352 [==============================] - 64s 27ms/step - loss: 35.0424
Epoch 194/500
2352/2352 [==============================] - 64s 27ms/step - loss: 35.0423
Epoch 195/500
2352/2352 [==============================] - 64s 27ms/step - loss: 35.0423
Epoch 196/500
2352/2352 [==============================] - 64s 27ms/step - loss: 35.0422
Epoch 197/500
2352/2352 [==========

2039/2352 [=========================>....] - ETA: 6s - loss: 34.4675

In [28]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [29]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=epochs, steps_per_epoch=steps_per_epoch, verbose=1)#, use_multiprocessing=True, workers=8)#, validation_data=val_generator(x_val))

Epoch 1/500
2352/2352 [==============================] - 22s 9ms/step - loss: 21.0663
Epoch 2/500
2352/2352 [==============================] - 20s 9ms/step - loss: 12.7610
Epoch 3/500
2352/2352 [==============================] - 20s 9ms/step - loss: 11.2392
Epoch 4/500
2352/2352 [==============================] - 20s 9ms/step - loss: 9.7048
Epoch 5/500
2352/2352 [==============================] - 21s 9ms/step - loss: 8.1129
Epoch 6/500
2352/2352 [==============================] - 21s 9ms/step - loss: 6.3650
Epoch 7/500
2352/2352 [==============================] - 21s 9ms/step - loss: 5.3914
Epoch 8/500
2352/2352 [==============================] - 20s 9ms/step - loss: 4.5152
Epoch 9/500
2352/2352 [==============================] - 21s 9ms/step - loss: 3.8071
Epoch 10/500
2352/2352 [==============================] - 20s 9ms/step - loss: 3.3941
Epoch 11/500
2352/2352 [==============================] - 20s 9ms/step - loss: 3.2027
Epoch 12/500
2352/2352 [==============================] - 20

2352/2352 [==============================] - 20s 9ms/step - loss: 0.1004
Epoch 97/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1334
Epoch 98/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1111
Epoch 99/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1181
Epoch 100/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1074
Epoch 101/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.1080
Epoch 102/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1125
Epoch 103/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1978
Epoch 104/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.1819
Epoch 105/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.2364
Epoch 106/500
2352/2352 [==============================] - 22s 9ms/step - loss: 0.1985
Epoch 107/500
2352/2352 [==============================] - 2

2352/2352 [==============================] - 21s 9ms/step - loss: 0.2068
Epoch 191/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0945
Epoch 192/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0392
Epoch 193/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0336
Epoch 194/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0503
Epoch 195/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0503
Epoch 196/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0544
Epoch 197/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0499
Epoch 198/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0508
Epoch 199/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0508
Epoch 200/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0476
Epoch 201/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 8ms/step - loss: 0.0374
Epoch 285/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0386
Epoch 286/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0513
Epoch 287/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0483
Epoch 288/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0288
Epoch 289/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0377
Epoch 290/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0397
Epoch 291/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0387
Epoch 292/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0347
Epoch 293/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0354
Epoch 294/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0359
Epoch 295/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 9ms/step - loss: 0.0279
Epoch 379/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0307
Epoch 380/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0261
Epoch 381/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0259
Epoch 382/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0322
Epoch 383/500
2352/2352 [==============================] - 22s 9ms/step - loss: 0.0278
Epoch 384/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0271
Epoch 385/500
2352/2352 [==============================] - 22s 9ms/step - loss: 0.0322
Epoch 386/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0267
Epoch 387/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0291
Epoch 388/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0271
Epoch 389/500
2352/2352 [==============================] 

2352/2352 [==============================] - 20s 9ms/step - loss: 0.0281
Epoch 473/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0240
Epoch 474/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0234
Epoch 475/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0258
Epoch 476/500
2352/2352 [==============================] - 20s 8ms/step - loss: 0.0217
Epoch 477/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0242
Epoch 478/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0239
Epoch 479/500
2352/2352 [==============================] - 20s 9ms/step - loss: 0.0234
Epoch 480/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0242
Epoch 481/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0312
Epoch 482/500
2352/2352 [==============================] - 21s 9ms/step - loss: 0.0199
Epoch 483/500
2352/2352 [==============================] 

In [ ]:
#lstm_autoencoder_250.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=steps_per_epoch, verbose=1)#, validation_data=val_generator(x_val))

In [ ]:
# model save
lstm_autoencoder_256.save('model/lstm_autoencoder256_bfs.h5')

In [ ]:
loaded_model = load_model('model/lstm_autoencoder256_bfs.h5')

In [37]:
loaded_model = lstm_autoencoder

In [44]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

2.1981039783470795


In [19]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [20]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])

In [21]:
latent_vector = np.array(latent_vector)

In [144]:
def printMatrix(matrix):
    length = int(max(max(matrix[:, 0]), max(matrix[:, 1])))
    adj = [[0 for i in range(length)] for i in range(length)]
    
    for m in matrix:
        (i, j, w) = m
        i = int(i) -1
        j = int(j) -1
        adj[i][j] = adj[j][i] = w
    for i in range(length):
        for j in range(length):
            print(str(adj[i][j]), end=',')
        print()
    print("===============")

In [213]:
# sequence to matrix
for index, x in enumerate(x_test):
    print("#", str(index))
    printMatrix(x)

# 0
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0,0,0,
0,0,18.05,0,0,0,0,32.09,0,
0,0,0,20.05,0,0,0,0,20.27,
0,0,0,0,0,32.09,0,0,9.01,
0,0,0,0,0,0,20.27,9.01,0,
# 1
0,8.79,11.79,0,0,0,
8.79,0,0,10.06,6.86,0,
11.79,0,0,0,0,7.86,
0,10.06,0,0,0,0,
0,6.86,0,0,0,0,
0,0,7.86,0,0,0,
# 2
0,3.3,6.3,0,0,
3.3,0,0,12.72,0,
6.3,0,0,0,23.87,
0,12.72,0,0,41.95,
0,0,23.87,41.95,0,
# 3
0,5.7,7.22,0,0,
5.7,0,0,12.22,0,
7.22,0,0,0,23.54,
0,12.22,0,0,42.39,
0,0,23.54,42.39,0,
# 4
0,12.52,17.52,28.77,0,0,0,0,0,
12.52,0,0,0,0,0,0,0,0,
17.52,0,0,0,2.53,4.53,0,0,0,
28.77,0,0,0,0,0,10.86,0,0,
0,0,2.53,0,0,0,0,0,0,
0,0,4.53,0,0,0,0,9.01,0,
0,0,0,10.86,0,0,0,0,10.01,
0,0,0,0,0,9.01,0,0,20.01,
0,0,0,0,0,0,10.01,20.01,0,
# 5
0,4.27,9.29,0,0,
4.27,0,0,12.29,0,
9.29,0,0,0,17.29,
0,12.29,0,0,3.95,
0,0,17.29,3.95,0,
# 6
0,10.16,7.92,9.56,0,0,0,0,0,
10.16,0,0,0,0,0,0,0,0,
7.92,0,0,0,8.17,18.05,0,0,0,
9.56,0,0,0,0,0,20.05,0,0,
0,0,8.17,0,0,0,0

In [171]:
standard = latent_vector[0]

In [223]:
import copy
dist = []
left = 100000
right = -10000
for index, v in enumerate(latent_vector[1:]):
    d = round(euclidean_distance(standard, v), 8)
    dist.append(d)
    if d > right:
        right= d
    if left > d:
        left = d
dist1 = copy.deepcopy(dist)
dist1.sort()
#print(dist, dist1)
for index, d in enumerate(dist1):
    print( (index+1, dist.index(d)+1), end='  ')

(1, 2)  (2, 38)  (3, 53)  (4, 5)  (5, 15)  (6, 44)  (7, 48)  (8, 21)  (9, 12)  (10, 43)  (11, 9)  (12, 18)  (13, 31)  (14, 59)  (15, 58)  (16, 4)  (17, 41)  (18, 27)  (19, 29)  (20, 33)  (21, 6)  (22, 3)  (23, 51)  (24, 35)  (25, 54)  (26, 55)  (27, 26)  (28, 62)  (29, 14)  (30, 46)  (31, 11)  (32, 45)  (33, 7)  (34, 50)  (35, 57)  (36, 8)  (37, 63)  (38, 60)  (39, 24)  (40, 52)  (41, 20)  (42, 13)  (43, 37)  (44, 23)  (45, 10)  (46, 1)  (47, 61)  (48, 42)  (49, 56)  (50, 34)  (51, 16)  (52, 32)  (53, 40)  (54, 39)  (55, 19)  (56, 17)  (57, 22)  (58, 49)  (59, 30)  (60, 36)  (61, 47)  (62, 28)  (63, 25)  

In [168]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum(a-b) ** 2)